In [15]:
%%writefile queue_job.sh
#!/bin/bash

MFD=$1
MHPE=$2
MFLD=$3
MGE=$4
INPUT=$5
DEVICE=$6
THRESHOLD=$7
OUTPUT=$8

mkdir -p ${OUTPUT}

exec 1>${OUTPUT}/stdout.log 2>${OUTPUT}/stderr.log

if echo "$DEVICE" | grep -q "FPGA"; then # if device passed in is FPGA, load bitstream to program FPGA
    #Environment variables and compilation for edge compute nodes with FPGAs
    export AOCL_BOARD_PACKAGE_ROOT=/opt/intel/openvino/bitstreams/a10_vision_design_sg2_bitstreams/BSP/a10_1150_sg2

    source /opt/altera/aocl-pro-rte/aclrte-linux64/init_opencl.sh
    
    aocl program acl0 /opt/intel/openvino/bitstreams/a10_vision_design_sg2_bitstreams/2020-2_PL2_FP16_MobileNet_Clamp.aocx

    export CL_CONTEXT_COMPILER_MODE_INTELFPGA=3
fi

python3 main.py  --model_face_detection ${MFD} \
                 --model_head_pose_estimation ${MHPE} \
                 --model_facial_landmarks_detection ${MFLD} \
                 --model_gaze_estimation ${MGE} \
                 --input_path ${INPUT} \
                 --device ${DEVICE} \
                 --threshold ${THRESHOLD} \

#cd ${OUTPUT}
#tar zcvf output.tgz *

Overwriting queue_job.sh
